In [1]:
from scrapybara import Scrapybara
from web_recorder import Recorder, Recording

In [2]:
SCRAPYBARA_API_KEY = "<api_key>"
client = Scrapybara(api_key=SCRAPYBARA_API_KEY)

instance = client.start_browser()

live_web_url = instance.get_stream_url().stream_url
cdp_url = instance.get_cdp_url().cdp_url
print(f"`🔍 Live View Link: {live_web_url}")

`🔍 Live View Link: https://api.proxy.scrapybara.com/v1/instance/s-cc2b3608/stream


In [3]:
cdp_url

'wss://proxy-api-102101747085.us-central1.run.app/v1/instance/s-cc2b3608/browser/ws'

In [4]:
recorder = Recorder(
  cdp_url=cdp_url,
)

recording = await recorder.record()

Task ID: 4ba7a19c-290f-4593-91d3-155b50c31b20


In [2]:
events_path = "recordings/events.json"

In [ ]:
recording.export(
  path=events_path,
)

In [3]:
recording = Recording.from_file(events_path)

In [4]:
len(recording.events)

228

In [5]:
await recording.replay() #replays on a local browser instance, can pass in a cdp url to replay on a remote browser

number of events 2


In [1]:
events_path = "recordings/events.json"
recording = Recording.from_file(events_path)

trajectory = await recording.get_trajectory()

len(trajectory.snapshots)

86

In [2]:
from web_recorder.recorder import ExportFormat, ExportConfig
events_path = "recordings/events.json"
recording = Recording.from_file(events_path)

await recording.export(
    path="./trajectories/trajectory.json",
    config=ExportConfig(
        format=ExportFormat.TRAJECTORY,
    ),
)

Successfully exported recording
